# Loan Duration Study - Utkarsh Gaikwad

# Imporing Data

In [84]:
from warnings import filterwarnings
filterwarnings('ignore')
import pandas as pd
training = pd.read_csv("D:/Etlhive class data/Project Loan 280822 Classification and Regression/Input/training_set.csv")
testing = pd.read_csv("D:/Etlhive class data/Project Loan 280822 Classification and Regression/Input/testing_set.csv")
training.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,NaN,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y


# Converting Credit History to Object datatype 

In [85]:
crd_hst = []
for i in range(len(training)):
    if training['Credit_History'][i]==1:
        crd_hst.append('Yes')
    elif training['Credit_History'][i]==0:
        crd_hst.append('No')
    else:
        crd_hst.append(None)
training['Credit_History']=crd_hst

In [86]:
crd_hst1 = []
for i in range(len(testing)):
    if testing['Credit_History'][i]==1:
        crd_hst1.append('Yes')
    elif testing['Credit_History'][i]==0:
        crd_hst1.append('No')
    else:
        crd_hst1.append(None)
testing['Credit_History']=crd_hst1

In [87]:
import import_ipynb
from PM3 import replacer
replacer(training)
replacer(testing)

Missing Values replaced in DataFrame
Missing Values replaced in DataFrame


# Defining X and Y

In [88]:
X = training.drop(labels=['Loan_ID','Loan_Amount_Term'],axis=1)
Y = training[['Loan_Amount_Term']]

In [89]:
from PM3 import catcon 
cat, con = catcon(X)

Categorical and Continuous variables appended


# Preprocessing

In [90]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X1 = pd.DataFrame(ss.fit_transform(X[con]),columns=con)
X2 = pd.get_dummies(X[cat])
Xnew = X1.join(X2)

In [91]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()

tg = {"n_neighbors":range(2,101,1),"weights":['uniform','distance']}
cv = GridSearchCV(knn,tg,scoring='neg_mean_absolute_error',cv=20)
cvmodel = cv.fit(Xnew,Y)
cvmodel.best_params_

{'n_neighbors': 64, 'weights': 'distance'}

In [92]:
model = cvmodel.best_estimator_
pred = model.predict(Xnew)
from sklearn.metrics import mean_squared_error, r2_score
err = mean_squared_error(pred,Y)
r2 = r2_score(pred,Y)
print('Mean Squared Error is      : ',round(err,4))
print('Root Mean Squared Error is : ',round(err**0.5,4))
print('R2 Score is                : ',round(r2,6))

Mean Squared Error is      :  0.0
Root Mean Squared Error is :  0.0
R2 Score is                :  1.0


# Filtering Out rejected Loan Applications and LoanDuration < 20 years or 240 months

In [93]:
X_no = training[(training['Loan_Amount_Term']<240) & (training['Loan_Status']=='N')]

In [94]:
X_no.index = range(len(X_no))
X_no

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001207,Male,Yes,0,Not Graduate,Yes,2609.0,3449.0,165.000000,180.0,No,Rural,N
1,LP001228,Male,No,0,Not Graduate,No,3200.0,2254.0,126.000000,180.0,No,Urban,N
2,LP001451,Male,Yes,1,Graduate,Yes,10513.0,3850.0,160.000000,180.0,No,Urban,N
3,LP001586,Male,Yes,3+,Not Graduate,No,3522.0,0.0,81.000000,180.0,Yes,Rural,N
4,LP001682,Male,Yes,3+,Not Graduate,No,3992.0,0.0,146.412162,180.0,Yes,Urban,N
5,LP001800,Male,Yes,1,Not Graduate,No,2510.0,1983.0,140.000000,180.0,Yes,Urban,N
6,LP001813,Male,No,0,Graduate,Yes,6050.0,4333.0,120.000000,180.0,Yes,Urban,N
7,LP001870,Female,No,1,Graduate,No,3481.0,0.0,155.000000,36.0,Yes,Semiurban,N
8,LP002205,Male,No,1,Graduate,No,3062.0,1987.0,111.000000,180.0,No,Urban,N
9,LP002288,Male,Yes,2,Not Graduate,No,2889.0,0.0,45.000000,180.0,No,Urban,N


In [95]:
X2 = X_no.drop(labels=['Loan_ID','Loan_Amount_Term'],axis=1)

In [96]:
cat, con = catcon(X2)

Categorical and Continuous variables appended


In [97]:
X21 = pd.DataFrame(ss.transform(X2[con]),columns=con)
X22 = pd.get_dummies(X2[cat])
Xnew2 = X21.join(X22)

In [98]:
Xnew2

,ApplicantIncome,CoapplicantIncome,LoanAmount,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,...,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Credit_History_No,Credit_History_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N
0,-0.458153,0.625241,0.221365,0,1,0,1,1,0,0,...,0,1,0,1,1,0,1,0,0,1
1,-0.361330,0.216533,-0.243092,0,1,1,0,1,0,0,...,0,1,1,0,1,0,0,0,1,1
2,0.836747,0.762389,0.161820,0,1,0,1,0,1,0,...,1,0,0,1,1,0,0,0,1,1
3,-0.308578,-0.554368,-0.779004,0,1,0,1,0,0,0,...,0,1,1,0,0,1,1,0,0,1
4,-0.231578,-0.554368,0.000000,0,1,0,1,0,0,0,...,0,1,1,0,0,1,0,0,1,1
5,-0.474372,0.123847,-0.076363,0,1,0,1,0,1,0,...,0,1,1,0,0,1,0,0,1,1
6,0.105581,0.927582,-0.314547,0,1,1,0,1,0,0,...,1,0,0,1,0,1,0,0,1,1
7,-0.315294,-0.554368,0.102274,1,0,1,0,0,1,0,...,1,0,1,0,0,1,0,1,0,1
8,-0.383939,0.125215,-0.421729,0,1,1,0,0,1,0,...,1,0,1,0,1,0,0,0,1,1
9,-0.412281,-0.554368,-1.207733,0,1,0,1,0,0,1,...,0,1,1,0,1,0,0,0,1,1


# Changing Loan_Status_N to Loan_Status_Y and predicting loan term 

In [99]:
Xnew2['Loan_Status_N']=0
Xnew2['Loan_Status_Y']=1

In [100]:
pred_term = model.predict(Xnew2)

In [101]:
X_no['Predicted_Loan_Duration']=pred_term

In [102]:
X_no

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Predicted_Loan_Duration
0,LP001207,Male,Yes,0,Not Graduate,Yes,2609.0,3449.0,165.000000,180.0,No,Rural,N,342.004287
1,LP001228,Male,No,0,Not Graduate,No,3200.0,2254.0,126.000000,180.0,No,Urban,N,341.552010
2,LP001451,Male,Yes,1,Graduate,Yes,10513.0,3850.0,160.000000,180.0,No,Urban,N,340.737413
3,LP001586,Male,Yes,3+,Not Graduate,No,3522.0,0.0,81.000000,180.0,Yes,Rural,N,334.853790
4,LP001682,Male,Yes,3+,Not Graduate,No,3992.0,0.0,146.412162,180.0,Yes,Urban,N,319.913992
5,LP001800,Male,Yes,1,Not Graduate,No,2510.0,1983.0,140.000000,180.0,Yes,Urban,N,335.563424
6,LP001813,Male,No,0,Graduate,Yes,6050.0,4333.0,120.000000,180.0,Yes,Urban,N,340.235805
7,LP001870,Female,No,1,Graduate,No,3481.0,0.0,155.000000,36.0,Yes,Semiurban,N,342.948918
8,LP002205,Male,No,1,Graduate,No,3062.0,1987.0,111.000000,180.0,No,Urban,N,338.504393
9,LP002288,Male,Yes,2,Not Graduate,No,2889.0,0.0,45.000000,180.0,No,Urban,N,325.027566


In [103]:
X_no.to_csv('D:/Etlhive class data/Project Loan 280822 Classification and Regression/Output/LoanTermPredictions.csv')